In [3]:
from transformers import pipeline 
from bs4 import BeautifulSoup
import requests

In [37]:
def get_website_text(URL):
    # Get the raw text from a URL
    r = requests.get(URL)
    soup = BeautifulSoup(r.text, 'html.parser')
    results = soup.find_all(['h2', 'p'])
    text  = [result.text for result in results]

    # Join all the texts together
    article = ' '.join(text)

    # Break up blog post into sentences
    article = article.replace('.', '.<eos>')
    article = article.replace('!', '!<eos>')
    article = article.replace('?', '?<eos>')
    sentences = article.split('<eos>')
    return sentences

def chunk_text(sentences):
    max_chunk = 500
    current_chunk = 0
    chunks = []

    # Create chunks with not more than 500 words
    for sentence in sentences:
        if len(chunks) == current_chunk + 1:
            if len(chunks[current_chunk]) + len(sentence.split(' ')) <= max_chunk:
                chunks[current_chunk].extend(sentence.split(' '))
            else:
                current_chunk += 1
                chunks.append(sentence.split(' '))
        
        else:
            print(current_chunk)
            chunks.append(sentence.split(' '))

    # Join chunks back together
    for chunk_id in range(len(chunks)):
        chunks[chunk_id] = ' '.join(chunks[chunk_id])

    return chunks

def load_and_summarize(chunks, max_length, min_lenght):
    # Load in the summarizer
    summarizer = pipeline('summarization')

    # Write a summary for each chunk
    res = summarizer(chunks, max_length=max_length, min_length=min_lenght, do_sample=False)

    # Append the individual chunks back together
    summary = ' '.join(summ['summary_text'] for summ in res)
    return summary

In [51]:
URL = "https://www.kern.ai/post/fine-tune-your-embeddings-for-better-similarity-search"

sentences = get_website_text(URL)
chunks = chunk_text(sentences)
summary = load_and_summarize(chunks, 60, 30)

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


0


In [52]:
summary

" This blog post will share our experience with fine-tuning sentence embeddings on a commonly available dataset using similarity learning . We additionally explore how this could benefit the labeling workflow in Kern refinery .  Quaterion is able to use different kinds of similarity information in order to fine-tune the embeddings . The goal is to learn a mapping from one embedding to another . For that we are going to use a pre-trained LLM as the encoder and add  The training details are covered by Quaterion for us, which uses PyTorch Lightning under the hood . The optimizer (we chose Adam) is specified in the model itself, we just need to call the fit method . The test data consists of the 9. 156 (al  Quaterion's neighbor-based similarity improved w. r. t. classification but this linear classification model did not find it easier to separate the classes from one another . We are working on methods to fine-tune embeddings leading to a better separation of classes in the 2"

In [53]:
with open('blog_summary.txt', 'w') as f:
    f.write(summary)